In [1]:
pip install crewai crewai_tools langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.8 MB

In [15]:
pip install utils


  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13906 sha256=aa41cac5c7c685aff06ee3e49e3e0fac39335092a119d6ab12abf66cab2c9c07
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [37]:
import os
os.environ['OPENAI_API_KEY'] = 'masked'

In [38]:
import warnings
warnings.filterwarnings('ignore')
import utils
import os
from openai import OpenAI


openai_api_key = get_openai_api_key()

os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'


In [39]:
from crewai import Agent, Task, Crew

Creating agents
1. Planner
2. writer
3. editor

In [40]:
planner = Agent(
    role = 'Content Planner',
    goal = "Plan engaging and factually accurate content for a blog post' on the {topic}",
    backstory = "You are an helpful ai assistant tasked to work on planning a blog article "
                "about the topic: {topic}."
                "audiance learn something interesting about the topic"
                "and make informed decisions. "
                " your work is the basis for the "
                "the content writer to write an article on this topic",
    allow_delegation = False,
    verbose = True
)


In [41]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [42]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

Creating Tasks

In [43]:
plan = Task(
    description = (
        "1. Prioritize the latest trends, key players, intester parties, "
          "and noteworthy news that has happened or about to happen on the {topic} . \n"
        "2. Identify the target audiance, considering "
          "thier intests and pain points. \n"
        "3. Develop a detailed content outline inculding an introduction, key points, applications and a call to action. \n"
        "4. Include SEO keywords and relavant data or sources if deemed necessary."
    ),
    expected_output = "A comprehensive content plan document with an outlie, audiance analysis, seo keywords and resurces.",
    agent = planner
    )


In [44]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [45]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [46]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [47]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, intester parties, and noteworthy news that has happened or about to happen on the Artificial Intelligence . 
2. Identify the target audiance, considering thier intests and pain points. 
3. Develop a detailed content outline inculding an introduction, key points, applications and a call to action. 
4. Include SEO keywords and relavant data or sources if deemed necessary.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan for Blog Post on Artificial Intelligence

Title: Unveiling the Power of Artificial Intelligence: Latest Trends and Applications

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and developments in AI
- Mention the key players in the AI industry and their contributions

Key Points:
1. Latest Trends in A

In [48]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Power of Artificial Intelligence: Latest Trends and Applications

**Introduction:**
Artificial Intelligence (AI) has revolutionized various industries, from healthcare to finance, with its innovative applications and advancements. Staying updated on the latest trends and developments in AI is crucial for professionals in tech, business, healthcare, finance, and marketing. Key players in the AI industry, such as Google, Amazon, and Microsoft, have been at the forefront of AI research, driving significant contributions to the field.

**Latest Trends in Artificial Intelligence:**
Machine Learning and Deep Learning have seen significant advancements in recent years, enabling AI systems to learn and improve from data without explicit programming. Natural Language Processing (NLP) breakthroughs have enhanced language understanding and generation capabilities in AI applications. Robotics and automation in AI have transformed industries, increasing efficiency and productivity. Ethical considerations in AI development have become critical, ensuring that AI systems are developed and deployed responsibly.

**Key Players in the AI Industry:**
Tech giants like Google, Amazon, and Microsoft continue to lead AI research, investing in cutting-edge technologies and solutions. Startups and innovators are disrupting the AI landscape, bringing fresh ideas and perspectives to the industry. Academic institutions and research centers play a crucial role in driving AI advancements, collaborating with industry partners to push the boundaries of AI technology.

**Applications of Artificial Intelligence:**
In healthcare, AI is revolutionizing disease diagnosis and personalized treatment, improving patient outcomes and healthcare delivery. In finance, AI is utilized for fraud detection and risk management, enhancing security and compliance measures. Marketing benefits from AI-powered analytics and customer segmentation, enabling targeted campaigns and personalized experiences. The automotive industry is embracing AI for self-driving cars and predictive maintenance, improving safety and efficiency on the roads.

**Call to Action:**
I encourage readers to explore further resources on AI trends and applications to stay informed and updated on the latest developments in the field. Feel free to share your thoughts and experiences with AI in the comments section below. For deeper insights, check out the relevant articles, reports, and case studies provided in the resources section.

By following this content plan, the blog post on Artificial Intelligence aims to engage professionals in tech, business, healthcare, finance, and marketing with valuable insights and resources. Keep up with rapidly evolving AI trends and understand complex concepts to implement AI solutions effectively in your respective fields.

Remember, the future of artificial intelligence is here, and it's essential to stay ahead of the curve to leverage its full potential.

In [49]:
topic = "ISB"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, intester parties, and noteworthy news that has happened or about to happen on the ISB . 
2. Identify the target audiance, considering thier intests and pain points. 
3. Develop a detailed content outline inculding an introduction, key points, applications and a call to action. 
4. Include SEO keywords and relavant data or sources if deemed necessary.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan for Blog Post on ISB:

1. Introduction:
- Brief overview of ISB (International Standards Bureau).
- Importance of adhering to international standards for businesses and organizations.
- Purpose of the blog post - to educate and inform readers about the significance of ISB.

2. Latest Trends and Noteworthy News on ISB:
- Highlight the recent developments in international standards set by ISB.
- Discuss key players in the indu

In [51]:
Markdown(result)

# The Significance of International Standards Bureau (ISB) in Today's Business World

In today's interconnected business environment, adherence to international standards is essential for the prosperity and longevity of businesses worldwide. The International Standards Bureau (ISB) plays a pivotal role in establishing and upholding these standards, ensuring that businesses operate ethically and effectively on a global scale. This blog post aims to explore the importance of ISB, highlight the latest trends and significant news within the industry, conduct an analysis of the target audience's interests and pain points, and offer valuable insights into the practical applications of ISB standards.

Recent advancements in international standards established by ISB have had a profound impact on how businesses function and collaborate with one another at a global level. Key stakeholders across diverse sectors have successfully integrated ISB standards into their operations, resulting in enhanced business processes and increased credibility within the market. As ISB continues to progress, anticipated revisions and updates in guidelines are set to further streamline business practices and ensure adherence to industry regulations.

The target audience for this blog post comprises business owners, managers, and decision-makers seeking to uphold international standards, enhance their business operations, and remain informed about industry regulations. These individuals often encounter difficulties in comprehending and implementing complex ISB standards, as well as a lack of awareness regarding the advantages of compliance. By addressing their challenges and offering valuable insights, our objective is to empower them to make informed decisions in their business endeavors.

In conclusion, the International Standards Bureau (ISB) holds a critical role in the business realm by establishing and maintaining international standards that promote ethical and efficient business conduct. By keeping abreast of the latest industry trends and developments, businesses can align themselves with ISB standards and reap the benefits of enhanced processes and credibility. Through our ongoing efforts to educate and enlighten our readers about the significance of ISB, we strive to foster proactive adherence to international standards and drive positive transformation within the business landscape. Remember, adherence is the cornerstone of success in today's global marketplace.